# Algorithms with Matrices

In this practice we are going to implement algorithms to work with matrices:
- solving simple equations
- inverting matrices
- decomposing matrices to LU decompositions

For these activities Gauss Elimination is our main tool. We will only work with matrices that do not require swapping the rows.

We will also perform QR decomposition using Gramm-Schmidt orthonormalization method

In [158]:
import numpy as np
import lovely_numpy as ln
import json_tricks 
import matplotlib.pyplot as plt

In [159]:
np.random.seed(0)


inputs = json_tricks.load('/workspaces/linear-algebra-part-1/.laborantum/texts/course/5. Gauss Elimination/Coding and Applying Gauss Elimination/inputs/inputs.json')
answer = {}

In [160]:
simple_cases = inputs[:50]  
pivoting_cases = inputs[50:80] 
nullspace_cases = inputs[80:]

In [161]:
# This cell is intentionally empty - debug code was removed

# Task 1. Implement Gauss Elimination Stage I

## 1.1 Gauss Elimination -- non-pivoted case
Firstly, your task here is to create an algorithm that will perfrom mutations of matrix $A$ that transform it to Upper-Triangular form $U$.

$$
\begin{bmatrix}
a_{11} & a_{12} & \dots & a_{1N} \\
a_{21} & a_{22} & \dots & a_{2N} \\
\vdots & \vdots & \ddots & \vdots \\
a_{N1} & a_{N2} & \dots & a_{NN}
\end{bmatrix}

\rightarrow

\begin{bmatrix}
1 & a_{12}^* & \dots & a_{1N}^* \\
0 & 1 & \dots & a_{2N}^* \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & 1
\end{bmatrix}
$$

Note that there should be unit diagonal after the 1 stage.

The same mutations for rows as you do to the matrix $A$ should be done to the rows of matrix $B$. 

What can matrix $B$ be? It can be:
- vector of right hand side $b$ (in that case we are solving the System of Linear Equations)
- identity matrix (in that case we invert matrix)
- actually, Stage I can be used as Stage II, and in that case right hand matrix can be some matrix from Stage I

## 1.2 Add pivoting

There is a flag `pivoting`. In case this flag is turned on, the Gauss Elimination should swap the current row with 
the row with maximal absolute value in the current column.

You should also perform the same swap with the matrix $B$ and also you should keep track of swaps with the matrix
$P$.

## 1.3 Account for the cases with null-space

In that case once in a while you will not be able to perform pivoting to evade the division-by-zero. In that case you
will find out that the diagonal element and all the elements below are all together less than some threshold value.

In that case just continue to the next column.

Keep in mind though that you have to advance only the column index, but the row index should not be increased. That
means that now your pivot is off the main diagonal.


In [162]:
def gauss_elimination(A, B, pivoting=False):
    A = A.copy().astype(float)
    B = np.atleast_2d(B)
    if B.shape[0] == 1 and A.shape[0] != 1:
        B = B.T
    B = B.astype(float)
    n_rows, n_cols = A.shape
    nB_rows, nB_cols = B.shape
    if nB_rows != n_rows:
        raise ValueError(f"B must have the same number of rows as A. Got {nB_rows} and {n_rows}")
    P = np.eye(n_rows)
    row = 0
    for col in range(n_cols):
        if row >= n_rows:
            break
        if pivoting:
            max_row = np.argmax(np.abs(A[row:, col])) + row
            if np.abs(A[max_row, col]) < 1e-12:
                continue
            if max_row != row:
                A[[row, max_row]] = A[[max_row, row]]
                B[[row, max_row]] = B[[max_row, row]]
                P[[row, max_row]] = P[[max_row, row]]
        if np.abs(A[row, col]) < 1e-12:
            continue
        A[row] = A[row] / A[row, col]
        B[row] = B[row] / A[row, col]
        for j in range(row + 1, n_rows):
            factor = A[j, col]
            A[j] -= factor * A[row]
            B[j] -= factor * B[row]
        row += 1
    return P, A, B


In [163]:
simple_cases = inputs[:50]  
pivoting_cases = inputs[50:80] 
nullspace_cases = inputs[80:]

answer['gauss_elimination_simple_cases'] = [
    gauss_elimination(A=input['A'], B=np.atleast_2d(input['b']).T, pivoting=False) for input in simple_cases
]

answer['gauss_elimination_pivoting_cases'] = [
    gauss_elimination(A=input['A'], B=np.atleast_2d(input['b']).T, pivoting=True) for input in pivoting_cases
]

answer['gauss_elimination_nullspace_cases'] = [
    gauss_elimination(A=input['A'], B=np.atleast_2d(input['b']).T, pivoting=True) for input in nullspace_cases
]

# Task 2: Solve System of Linear Equations (SLE)

Now we have the engine to solve a lot of things. We will start with solving Systems of Linear Equations (SLEs).

For the SLE given by matrix $A$ and right-hand vector $\mathbf b$, implement a function that solves this SLE.

You will need to use twice Gauss Elimination Algorithm

1. Apply it to matrix $A$ and to vector $\mathbf b$ as matrix $B$ as Gauss Elimintaion I stage
2. For Gauss Elimination II stage you can use the same algorithm by just applying it to reversed matrix 

    (by enumerating equations and variables in reversed order, we can rewrite the matrix $U$ in low-triangle form $L$). 

    Something has to be done to the right-hand side vector too
3. Form the answer from output of Gauss Elimination Stage II

In [164]:
def solve_equation(A, b):
    # Ensure b is a column vector
    b = np.atleast_2d(b).T
    P1, U, B1 = gauss_elimination(A, b, pivoting=True)
    P2, L, B2 = gauss_elimination(U[::-1, ::-1], B1[::-1], pivoting=False)
    x = B2[::-1].flatten()
    return x

You can always check that your solution is correct by subtracting

$A \mathbf x - \mathbf b$, which should give you $\mathbf 0$

By the way, matrix product, such as $A \mathbf x$ in numpy is done using `A @ x` operation

In [165]:
answer['solutions_simple_cases'] = [
    solve_equation(A=input['A'], b=input['b']) for input in simple_cases
]

answer['solution_pivoting_cases'] = [
    solve_equation(A=input['A'], b=input['b']) for input in pivoting_cases
]

# This should be close to zero
print("=== SIMPLE CASES ===")
for index in range(len(simple_cases)):
    print(np.abs(simple_cases[index]['A'] @ answer['solutions_simple_cases'][index] - simple_cases[index]['b']).sum())

print("=== PIVOTING CASES ===")
for index in range(len(pivoting_cases)):
    print(np.abs(pivoting_cases[index]['A'] @ answer['solution_pivoting_cases'][index] - pivoting_cases[index]['b']).sum())

=== SIMPLE CASES ===
3186019.804742055
153851.02288230736
6240125.679339603
515985.1982894236
0.0
6621659.037070073
380697.33192337805
198383.4878830859
0.0
0.0
35862.57986310247
593.3333333333338
333.97664141413895
652786.5129958093
0.0
0.0
1566.0500000000002
370.6666666666667
3252460.3775044023
102111.54651300423
1099.75
6712.8630434785155
117927.52867696962
2546.7911010714715
15144.568888888885
0.0
10249523.307583332
798954.6896837119
121196504.1475029
3877.6725905673356
50872.586156092104
86266.10699767689
76.5
28218.84931351262
7066937.552488181
659338.3632135838
7590.3
100207.17034528223
161722.97224368458
262030.76924312668
47382687.86213684
162635.23228870006
2219960.478687525
1390411.3475118137
59.0
129761.02728395106
0.0
0.0
1646.320987654346
268729.40983655717
=== PIVOTING CASES ===
0.0
276.0
6004.121527777794
27542.55648796477
633568.5287693467
15877046.292817537
314088.317980416
6285.081481481482
107440.46644199925
172015.76443428063
64460.001460888634
84396.37124242629
0.

# Task 3: Find inverse of $A$

Use the same approach as with solving SLE, but now instead of $\mathbf b$ for right hand side, insert corresponding Identity matrix $I$ as input into Gaussian Elimination algorithm

Identitiy matrix of size $N \times N$ is built using `numpy.eye(N)` 

In [166]:
def find_inverse(A):
    n = A.shape[0]
    I = np.eye(n)
    P1, U, B1 = gauss_elimination(A, I, pivoting=True)
    P2, L, B2 = gauss_elimination(U[::-1, ::-1], B1[::-1, ::-1], pivoting=False)
    inverse = B2[::-1, ::-1]
    return inverse

You can always check that the matrix that you have found is indeed inverse to $A$ as $A^{-1} A = I$

In [167]:
answer['simple_inverses'] = [
    find_inverse(input['A']) for input in simple_cases
]

answer['pivoting_inverses'] = [
    find_inverse(input['A']) for input in pivoting_cases
]

# This should be I
print("=== SIMPLE CASES ===")
for index in range(len(simple_cases)):
    print(answer['simple_inverses'][index] @ simple_cases[index]['A'])

print("=== PIVOTING CASES ===")
for index in range(len(pivoting_cases)):
    print(answer['pivoting_inverses'][index] @ pivoting_cases[index]['A'])

=== SIMPLE CASES ===
[[-4.21602578e+06  3.63191168e+07  5.70495816e+07  7.35877414e+07
   2.57189099e+07 -5.19228442e+07]
 [-4.17715633e+05  3.59842254e+06  5.65241268e+06  7.29094455e+06
   2.54821761e+06 -5.14438724e+06]
 [-3.81627761e+04  3.28741911e+05  5.16536845e+05  6.66143077e+05
   2.32901396e+05 -4.69929302e+05]
 [-6.07397674e+03  5.23266995e+04  8.21678402e+04  1.06009992e+05
   3.70358406e+04 -7.48163094e+04]
 [-2.18252560e+01  2.05771793e+02  1.10092693e+02  3.28071628e+02
  -2.85185982e+00 -3.62771597e+02]
 [-4.02836566e+02  3.46438840e+03  5.51217628e+03  7.04871857e+03
   2.50231218e+03 -4.93014324e+03]]
[[ 1.98017105e+05  4.23319496e+04 -2.26093828e+06 -4.46497220e+03
  -2.17806184e+06]
 [-5.43297359e+04 -1.16094510e+04  6.20359726e+05  1.10748791e+03
   5.97651710e+05]
 [ 1.61695445e+04  3.45727648e+03 -1.84618862e+05 -3.78922882e+02
  -1.77847736e+05]
 [ 1.67162303e+03  3.57623799e+02 -1.90848639e+04 -4.46593612e+01
  -1.83820923e+04]
 [ 1.59520217e+02  3.37193191e+0

# Task 4: LU decomposition

You can find LU decomposition of the matrix $A$ using Gauss Elimination. Again, it is done using 2 stages of Gauss Elimination:

1. $A \mathbf x = I \mathbf b \rightarrow U \mathbf x = L^* \mathbf b$

2. $I U \mathbf x = L^* \mathbf b \rightarrow L U \mathbf x = I \mathbf b$

But in case you are doing pivoted version, it is a little more tricky. You have to modify somehow the matrix $L$
after the first stage of Gauss Elimination.

In [168]:
def find_plu(A, pivoting=True):
    A = A.copy().astype(float)
    n = A.shape[0]
    I = np.eye(n)
    P, U, L_inv = gauss_elimination(A, I, pivoting=pivoting)
    L_inv = L_inv @ P.T
    _, _, L = gauss_elimination(L_inv, I, pivoting=False)
    return P.T, L, U

You can always check that your LU decomposition works as

$LU - A = O$ (a zero matrix)

In [169]:
answer['simple_PLU'] = [
    find_plu(input['A'], pivoting=False) for input in simple_cases
]

answer['pivoting_PLU'] = [
    find_plu(input['A'], pivoting=True) for input in pivoting_cases
]

answer['nullspace_PLU'] = [
    find_plu(input['A'], pivoting=True) for input in nullspace_cases
]


# This should be close to zero
print("=== SIMPLE CASES ===")
for index in range(len(simple_cases)):
    print(np.abs(answer['simple_PLU'][index][0] @ answer['simple_PLU'][index][1] @ answer['simple_PLU'][index][2] - simple_cases[index]['A']).sum())

print("=== PIVOTING CASES ===")
for index in range(len(pivoting_cases)):
    print(np.abs(answer['pivoting_PLU'][index][0] @ answer['pivoting_PLU'][index][1] @ answer['pivoting_PLU'][index][2] - pivoting_cases[index]['A']).sum())

print("=== NULLSPACE CASES ===")
for index in range(len(nullspace_cases)):
    print(np.abs(answer['nullspace_PLU'][index][0] @ answer['nullspace_PLU'][index][1] @ answer['nullspace_PLU'][index][2] - nullspace_cases[index]['A']).sum())

=== SIMPLE CASES ===
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
=== PIVOTING CASES ===
0.0
11.333333333333334
203.84583333333336
366.56417291737193
849.4002485536589
2146.6874892354867
659.7167577413479
96.44444444444444
618.0323188734955
1115.9976694143363
194.2446193010877
446.3554059907804
0.0
57.0
364.06833964646574
427.53830726179143
1821.5639852551747
169.9134963768116
208.15286195286197
1388.4406839270223
1748.0493958777038
1088.675392238129
2301.0814472743014
1955.2801542886214
1038.6317184871348
0.0
863.6730137248912
1539.0312482530817
2725.7068980517492
1112.188549130791
=== NULLSPACE CASES ===
228.44604628373284
579.6085854269342
30.0
1945.658582528427
2400.487720511154
1029.1581781226078
0.0
852.1103520928856
171.61927144535844
2253.021457359425
571.8149312614037
561.6376277056446
91.54761904761907
331.78315387760665
35

# Finding NullSpace

Now find null-space of the matrix using Gauss Elimination. To do so, analyze the matrix $U$ that appears after Gauss Elimination. Observing this matrix
it is easy to write out the vectors of Null Space.

Note that variable `eps` stands for tolerance. If a value deviates from 0 more than on `eps`, we consider it to be different from `0`.

In [170]:
def find_null_space(A, pivoting=True, eps=1e-12):
    """
    Return an (n x k) matrix whose columns span Null(A), i.e. { x : A x = 0 }.
    """
    A = A.astype(float)
    _, U, _ = gauss_elimination(A, np.eye(A.shape[0]), pivoting=pivoting)

    n_rows, n_cols = U.shape
    
    # Find pivot columns (columns with leading 1s)
    pivot_cols = []
    for i in range(min(n_rows, n_cols)):
        if i < n_rows and np.abs(U[i, i]) >= eps:
            pivot_cols.append(i)
    
    # Find free variables (non-pivot columns)
    free_vars = [j for j in range(n_cols) if j not in pivot_cols]
    
    if len(free_vars) == 0:
        # No free variables, null space is empty
        return np.empty((n_cols, 0))
    
    # Create basis vectors for null space
    null_vectors = []
    for free_var in free_vars:
        x = np.zeros(n_cols)
        x[free_var] = 1
        
        # Back substitution to find values of pivot variables
        for i in range(len(pivot_cols) - 1, -1, -1):
            pivot_row = i
            pivot_col = pivot_cols[i]

            if pivot_row < n_rows and np.abs(U[pivot_row, pivot_col]) >= eps:
                # x[pivot_col] = -sum(U[pivot_row, j] * x[j] for j in range(pivot_col + 1, n_cols)) / U[pivot_row, pivot_col]
                sum_val = 0
                for j in range(pivot_col + 1, n_cols):
                    sum_val += U[pivot_row, j] * x[j]
                x[pivot_col] = -sum_val / U[pivot_row, pivot_col]
        
        null_vectors.append(x)
    
    # Convert to matrix with proper shape
    if len(null_vectors) == 0:
        return np.empty((n_cols, 0))
    else:
        return np.column_stack(null_vectors)

In [171]:
answer['nullspace'] = [
    find_null_space(input['A'], pivoting=True) for input in nullspace_cases
]

for index in range(len(answer['nullspace'])):
    ns = answer['nullspace'][index]
    print(f"Case {index}: shape={ns.shape}")
    if ns.ndim == 2 and ns.shape[1] > 0:
        result = nullspace_cases[index]['A'] @ ns
        print(f"  Result shape: {result.shape}, max abs value: {np.abs(result).max()}")
    else:
        print(f"  Empty null space or invalid shape")

Case 0: shape=(4, 0)
  Empty null space or invalid shape
Case 1: shape=(6, 0)
  Empty null space or invalid shape
Case 2: shape=(3, 0)
  Empty null space or invalid shape
Case 3: shape=(8, 0)
  Empty null space or invalid shape
Case 4: shape=(9, 0)
  Empty null space or invalid shape
Case 5: shape=(7, 0)
  Empty null space or invalid shape
Case 6: shape=(1, 0)
  Empty null space or invalid shape
Case 7: shape=(7, 0)
  Empty null space or invalid shape
Case 8: shape=(4, 0)
  Empty null space or invalid shape
Case 9: shape=(8, 0)
  Empty null space or invalid shape
Case 10: shape=(5, 0)
  Empty null space or invalid shape
Case 11: shape=(6, 0)
  Empty null space or invalid shape
Case 12: shape=(3, 0)
  Empty null space or invalid shape
Case 13: shape=(5, 0)
  Empty null space or invalid shape
Case 14: shape=(5, 0)
  Empty null space or invalid shape
Case 15: shape=(2, 0)
  Empty null space or invalid shape
Case 16: shape=(2, 0)
  Empty null space or invalid shape
Case 17: shape=(5, 0)
  

# QR decomposition

Now again we will consider matrix $A^T$ in context of SLE:

$A^T \mathbf x = I \mathbf b$

Out of matrix $A$ we can make an orthonormal matrix $Q$ by orthonoramlizing the rows, this process will turn the $I$ matrix on the right to lower-triangular matrix $L^*$:

$A^T \mathbf x = I \mathbf b \rightarrow Q^T \mathbf x = L^* \mathbf b$

After that we should invert matrix $L^*$ and we will get:

$I Q^T \mathbf x = L^* \mathbf b \rightarrow L Q^T \mathbf x = \mathbf b$

If we transpose that matrix, we will get  QR decomposition of matrix $A$:

$A = Q R,$

where $R$ is upper-triangular matrix.

The task is to:
1. Implement Gramm-Schmidt orthonormalization procedure
2. Combining Gramm-Schmidt algorithm and Gaussian Algorithm, implement QR decomposition

# Task 5: Gramm-Schmidt Algorithm for Matrices

Implement Gramm-Schmidt process for **rows** of matrix $A$ (and do the same transforms to matrix $B$)

In [172]:
def gramm_schmidt(A, B):
    A = A.copy().astype(float)
    B = np.atleast_2d(B)
    if B.shape[0] == 1 and A.shape[0] != 1:
        B = B.T
    # Gramm-Schmidt process on rows of A
    for i in range(A.shape[0]):
        for j in range(i):
            factor = np.dot(A[i, :], A[j, :])
            A[i, :] -= factor * A[j, :]
            B[i, :] -= factor * B[j, :]
        norm = np.linalg.norm(A[i, :])
        if norm > 1e-12:
            A[i, :] /= norm
            B[i, :] /= norm
    return A, B

In [173]:
answer['gramm_schmidt_simple_cases'] = [
    gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in simple_cases
]

answer['gramm_schmidt_pivoting_cases'] = [
    gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in pivoting_cases
]

# answer['gramm_schmidt_nullspace_cases'] = [
    # gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in nullspace_cases
# ]

print("===== Should be close to 1s: =====")
print((answer['gramm_schmidt_simple_cases'][0][0] ** 2).sum(axis=1))
print()
print("===== Should be close to L: =====")
print(answer['gramm_schmidt_simple_cases'][0][1])

===== Should be close to 1s: =====
[1. 1. 1. 1. 1. 1.]

===== Should be close to L: =====
[[ 6.81994339e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-5.98077634e-01  7.49776626e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.33077525e+00 -8.45422150e-01  2.02271943e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.04755867e+01  2.24310668e+00 -9.04248636e-01  1.48799679e-01
   0.00000000e+00  0.00000000e+00]
 [-2.34404423e+02  2.60368931e+01 -8.83352031e+00  1.22899625e+00
   1.77501392e-01  0.00000000e+00]
 [-5.09021016e+08  5.64351750e+07 -1.93541498e+07  2.70677384e+06
   3.70898093e+05  3.66056342e+04]]


# Task 6: QR decomposition

Implement QR decomposition algorithm. Note that we work on rows on matrix $A$ with our Gauss and Gramm-Schmidt algorithms, but QR decomposition is defined for column matrix 

In [174]:
def find_qr(A):
    # Work with A^T for row operations
    A_T = A.T.copy()
    n = A_T.shape[0]
    I = np.eye(n)
    
    # Apply Gramm-Smidt to orthonormalize rows of A^T
    Q_T, L_inv = gramm_schmidt(A_T, I)
    
    # Invert L_inv to get L by solving L_inv * L = I
    _, _, L = gauss_elimination(L_inv, I, pivoting=False)
    
    # Q = Q_T^T, R = L^T (since we worked with A^T)
    Q = Q_T.T
    R = L.T
    
    return Q, R

In [175]:
answer['QR_simple_cases'] = [
    find_qr(input['A']) for input in simple_cases
]

print("===== Should be close to O: =====")
print(answer['QR_simple_cases'][0][0] @ answer['QR_simple_cases'][0][1] - simple_cases[0]['A'])

===== Should be close to O: =====
[[-9.94475138e-01  1.00405553e+01  6.60409631e-01  2.22647243e+01
   5.13069621e+01  1.31492198e+07]
 [-7.95580110e+00  7.93280496e+01  1.53416895e+01  1.85412489e+02
   4.20016120e+02  1.06439073e+08]
 [ 2.98342541e+00 -3.41072363e+01  3.72632899e+01 -3.13316435e+01
  -1.13875091e+02 -3.47316753e+07]
 [ 9.94475138e-01 -1.90080889e+01  8.39305165e+01  8.02281340e+01
   5.27376374e+01 -1.54663804e+06]
 [-4.97237569e+00  6.01667027e+01 -1.05195124e+02  9.46800883e+01
   1.24305096e+02  5.57161541e+07]
 [-8.95027624e+00  8.13974639e+01  9.25128748e+01  2.93755512e+02
   5.41744549e+02  1.29081915e+08]]


In [176]:
json_tricks.dump(answer, '.answer.json')

'{"gauss_elimination_simple_cases": [[{"__ndarray__": [[1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]], "dtype": "float64", "shape": [6, 6], "Corder": true}, {"__ndarray__": [[1.0, -10.0, 0.0, -7.0, 1.0, 8.0], [0.0, 1.0, -10.0, -6.0, -5.0, -4.0], [0.0, 0.0, 1.0, -6.0, -1.0, 0.0], [0.0, 0.0, 0.0, 1.0, -7.0, -4.0], [0.0, 0.0, 0.0, 0.0, 1.0, -7.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0]], "dtype": "float64", "shape": [6, 6], "Corder": true}, {"__ndarray__": [[147.0], [3.0], [40.0], [-8.0], [-30.0], [4.0]], "dtype": "float64", "shape": [6, 1], "Corder": true}], [{"__ndarray__": [[1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0]], "dtype": "float64", "shape": [5, 5], "Corder": true}, {"__ndarray__": [[1.0, 1.0, -8.0, -8.0, -7.0], [0.0, 1.0, 4.0, -7.0, 7.0], [0.0, 0.0, 1.0, -